# LLM Chatbot Behavior With Prompt and no context document

### **Goals for the MEPO LLM Chat Bot**
This notebook is designed to explore the behavior and responses of an LLM chatbot, specifically when it operates without a document as context but with a prompt guiding its behavior. Through this exercise, you will critically analyze how these AI systems function and consider the implications of their usage.

### **Context and Importance**

As AI systems become increasingly integrated into our daily lives, it's crucial to understand how they operate and the implications of their use. This notebook encourages you to critically think about the role of AI chatbots and what it means to engage with them meaningfully.

###  **Dangers of AI Reliance**

While AI chatbots can be incredibly useful, they also come with potential pitfalls. Over-reliance on these systems can introduce risks such as:
- **Bias:** AI models can perpetuate or amplify existing biases present in their training data.
- **Lack of Transparency:** Understanding how AI systems make decisions can be challenging, leading to a "black box" problem.
- **Amplification of Inequalities:** AI may inadvertently reinforce societal inequalities.

###  **Importance of Attention and Critical Thinking**

Human attention and critical thinking are paramount when using AI systems. Relying too heavily on AI can lead to decreased engagement and a decline in essential analytical skills. This notebook will emphasize the importance of maintaining a balance between AI assistance and human judgment.

###  **Responsible AI Usage**

This section addresses the ethical considerations and guidelines for responsible AI adoption. It will help you answer the question, "How should I use these AI systems?" by providing a framework for ethical and mindful interaction with AI.

### **Choice of LLM (Open Data)**

The notebook will also explore the rationale behind choosing an open-data LLM. Open-data models promote transparency, accessibility, and the potential for community-driven development, making them a valuable tool for education and innovation.

---

# Purpose of the Notebook

The primary purpose of this notebook is to guide you through the necessary steps to set up and interact with the LLaMA-2 model. By following along, you will gain insights into how these models work, and how they respond when provided with specific prompts in the absence of document-based context. This hands-on experience will help in forming design solutions and understanding the broader implications of AI systems.



# Setup and Query LLaMA-2 Model
This notebook will guide you through installing required libraries, setting up the LLaMA-2 model, and querying it using natural language.

## Install Required Libraries
We need to install the necessary libraries for PyTorch, TorchVision, and Torchaudio. Additionally, we'll install other dependencies required for running the LLaMA-2 model and handling document embeddings.

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade
!pip install langchain einops accelerate transformers bitsandbytes scipy
!pip install xformers sentencepiece
!pip install llama-index==0.10.12 llama_hub==0.0.19
!pip install llama-index-llms-huggingface
!pip install sentence-transformers
!pip install PyPDF2#dont need
!pip install PyMuPDF
!pip install --upgrade langchain llama-index
!pip install -U langchain-community
!pip install gradio==3.32.0
!pip install transformers
!pip install --upgrade gradio


Looking in indexes: https://download.pytorch.org/whl/cu117
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 77.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 67.4 MB/s eta 0:00:00
  Created wheel for lit: filename=lit-15.0.7-py3-none-any.whl size=89990 sha256=662aa6ad0e511c5e5bf9c3a245fa03a58d1fa7c1fe9ea094176d87444b6ea169
  Stored in directory: /root/.cache/pip/wheels/27/2c/b6/3ed2983b1b4

## Import Required Libraries
Next, we'll import the necessary libraries for tokenization, model setup, and text generation.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.llms.huggingface import HuggingFaceLLM

from llama_index.legacy.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings # This import should now work
from sentence_transformers import SentenceTransformer

from llama_index.core import set_global_service_context, ServiceContext

from llama_index.core import VectorStoreIndex, download_loader, Document # Import Document
from pathlib import Path
import fitz  # PyMuPDF
import gradio as gr


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warn

## Define Model and Tokenizer
We'll define the model name and the authentication token required to access the LLaMA-2 model from Hugging Face.

In [3]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
token_file = open("HF_TOKEN.txt")
auth_token = token_file.readline().strip();



tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='./model/', token=auth_token)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Load the Model
Now, we'll load the LLaMA-2 model using the previously defined name and authentication token. We'll also set some model parameters.

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir='./model/',
                                             use_auth_token=auth_token,
                                             torch_dtype=torch.float16,
                                             rope_scaling={"type": "dynamic", "factor": 2},
                                             load_in_8bit=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

## Create System and Query Prompts
Define the system prompt and query wrapper prompt to guide the LLaMA-2 model.

In [5]:
system_prompt = """<s>[INST] <<SYS>>
be concise and straightforward
<</SYS>>"""
# Throw together the query wrapper
query_wrapper_prompt = SimpleInputPrompt("{query_str} [/INST]")

#Create the Gradio Interface
Gradio provides a simple way to create a web-based interface for interacting with the model:

In [11]:
def query_llama2(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_length=512, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

iface = gr.Interface(fn=query_llama2,
                     inputs="text",
                     outputs="text",
                     title="LLaMA-2 Chat Bot",
                     description="Interact with the LLaMA-2 model to explore various topics.")

iface.launch(debug=True,share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7070b0b957cbc5249c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7070b0b957cbc5249c.gradio.live


In [8]:
query_llama2("How could a telemedicine system have been implemented using the technology available during the 1910s to combat the Spanish Flu pandemic?")

'How could a telemedicine system have been implemented using the technology available during the 1910s to combat the Spanish Flu pandemic?\n\nThe Spanish Flu pandemic of 1918-1919 was a global outbreak that infected an estimated 500 million people and killed an estimated 20 million worldwide. Given the limited technology available at the time, it is unlikely that a telemedicine system could have been implemented to combat the pandemic.\n\nDuring the 1910s, telecommunications technology was still in its infancy. While the telegraph had been in use for decades, the telephone was still a relatively new technology, and radio communication was just beginning to be developed. Internet connectivity and video conferencing were non-existent, and the use of computers and software was limited to a few specialized applications.\n\nIn addition, medical technology was also limited during this time period. Diagnostic tools such as X-rays and blood tests were still in their infancy, and many medical p

In [9]:
query_llama2("What affordable and sustainable housing solutions could have been developed during the Great Depression to address widespread homelessness?")

'What affordable and sustainable housing solutions could have been developed during the Great Depression to address widespread homelessness?\n\nDuring the Great Depression, widespread homelessness was a major social and economic issue in the United States. In response, the federal government and local organizations developed a range of affordable and sustainable housing solutions to address this problem. Here are some of the solutions that could have been developed during this time:\n\n1. Row Housing: Row houses were a common type of housing during the Great Depression, particularly in urban areas. These houses were often small, affordable, and efficient, with multiple units arranged in a row. Row houses were a sustainable housing solution because they were compact and used less land than single-family homes, making them more affordable and environmentally friendly.\n2. Co-operative Housing: Co-operative housing was another solution that emerged during the Great Depression. This type o

In [10]:
query_llama2("How might renewable energy technologies like solar or wind power have been developed in the 1970s to address the energy crisis?")

'How might renewable energy technologies like solar or wind power have been developed in the 1970s to address the energy crisis?\n\nIn the 1970s, the world was facing an energy crisis due to the rapid depletion of fossil fuels and the environmental impact of their use. Renewable energy technologies like solar and wind power were in their infancy, but they had the potential to address this crisis. Here are some ways that renewable energy technologies might have been developed in the 1970s to address the energy crisis:\n\n1. Government Support: Governments around the world could have provided more support for the development of renewable energy technologies through funding research and development, providing tax incentives, and creating regulatory frameworks that encouraged the use of renewable energy.\n2. Public Education and Awareness: Public education and awareness campaigns could have been launched to inform people about the benefits of renewable energy and the importance of reducing